In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [76]:
data = pd.read_csv('C:\\Users\\MDW\\Desktop\\dhfwhef.csv')

In [80]:
data=data.drop_duplicates()
data.head(5)

,user_id,item_id,timestamp
0,1,phone1,1/1/2022 10:00
1,1,case1,1/1/2022 10:05
2,1,charger1,1/1/2022 10:10
15,1,earphones1,1/1/2022 10:15
16,1,screen protector1,1/1/2022 10:20


In [82]:
data = data.sort_values(by=['user_id', 'timestamp'])
data.head(10)

,user_id,item_id,timestamp
0,1,phone1,1/1/2022 10:00
1,1,case1,1/1/2022 10:05
2,1,charger1,1/1/2022 10:10
15,1,earphones1,1/1/2022 10:15
16,1,screen protector1,1/1/2022 10:20
3,2,phone2,1/1/2022 10:00
4,2,case2,1/1/2022 10:05
5,2,charger2,1/1/2022 10:10
6,2,earphones2,1/1/2022 10:15
21,2,screen protector2,1/1/2022 10:20


In [83]:
data.shape

(21, 3)

In [84]:
label_encoder = LabelEncoder()
data['item_id'] = label_encoder.fit_transform(data['item_id'])
data.head(10)

,user_id,item_id,timestamp
0,1,14,1/1/2022 10:00
1,1,2,1/1/2022 10:05
2,1,5,1/1/2022 10:10
15,1,8,1/1/2022 10:15
16,1,17,1/1/2022 10:20
3,2,15,1/1/2022 10:00
4,2,3,1/1/2022 10:05
5,2,6,1/1/2022 10:10
6,2,9,1/1/2022 10:15
21,2,18,1/1/2022 10:20


In [85]:
sequences = data.groupby('user_id')['item_id'].apply(list)
sequences

user_id
1                    [14, 2, 5, 8, 17]
2             [15, 3, 6, 9, 18, 12, 0]
3    [16, 4, 7, 10, 19, 13, 1, 20, 11]
Name: item_id, dtype: object

In [86]:
max_sequence_length = 10
X = pad_sequences(sequences, maxlen=max_sequence_length, padding='pre', truncating='pre')
y = np.array([seq[-1] for seq in sequences])
X

array([[ 0,  0,  0,  0,  0, 14,  2,  5,  8, 17],
       [ 0,  0,  0, 15,  3,  6,  9, 18, 12,  0],
       [ 0, 16,  4,  7, 10, 19, 13,  1, 20, 11]])

In [87]:
y

array([17,  0, 11])

In [88]:
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
X_test

array([[ 0, 16,  4,  7, 10, 19, 13,  1, 20, 11]])

In [89]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_sequence_length, 1)))
model.add(Dropout(0.2))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [90]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [91]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [92]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)
model.save('sequential_recommendation_model.h5')

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 3.2680 - accuracy: 0.0000e+00 - val_loss: 4.2099 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 87ms/step - loss: 1.4128 - accuracy: 0.5000 - val_loss: 5.4873 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 58ms/step - loss: 0.6809 - accuracy: 0.5000 - val_loss: 6.7520 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 0.3153 - accuracy: 1.0000 - val_loss: 7.7972 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 0.3132 - accuracy: 1.0000 - val_loss: 8.5680 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1056 - accuracy: 1.0000 - val_loss: 8.9206 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 0.1532 - accuracy: 1.0000 - val_loss: 9.4103 - val_accuracy: 0

1/1 [==============================] - 0s 32ms/step - loss: 9.1251e-05 - accuracy: 1.0000 - val_loss: 12.1760 - val_accuracy: 0.0000e+00
Epoch 57/100
1/1 [==============================] - 0s 29ms/step - loss: 2.3392e-04 - accuracy: 1.0000 - val_loss: 12.1860 - val_accuracy: 0.0000e+00
Epoch 58/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8714e-04 - accuracy: 1.0000 - val_loss: 12.1961 - val_accuracy: 0.0000e+00
Epoch 59/100
1/1 [==============================] - 0s 33ms/step - loss: 1.0973e-04 - accuracy: 1.0000 - val_loss: 12.2058 - val_accuracy: 0.0000e+00
Epoch 60/100
1/1 [==============================] - 0s 19ms/step - loss: 2.2354e-04 - accuracy: 1.0000 - val_loss: 12.2150 - val_accuracy: 0.0000e+00
Epoch 61/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2546e-04 - accuracy: 1.0000 - val_loss: 12.2239 - val_accuracy: 0.0000e+00
Epoch 62/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3315e-04 - accuracy: 1.0000 - val_loss

In [93]:
new_user_data = pd.DataFrame({'user_id': [4], 'item_id': ['phone1'], 'timestamp': ['2022-01-01 10:00:00']})
new_user_data

,user_id,item_id,timestamp
0,4,phone1,2022-01-01 10:00:00


In [94]:
new_user_data['item_id'] = label_encoder.transform(new_user_data['item_id'])
new_user_data

,user_id,item_id,timestamp
0,4,14,2022-01-01 10:00:00


In [95]:
new_user_sequence = new_user_data['item_id'].tolist()
new_user_sequence

[14]

In [96]:
new_user_sequence = pad_sequences([new_user_sequence], maxlen=max_sequence_length, padding='pre', truncating='pre')
new_user_sequence = new_user_sequence.reshape((new_user_sequence.shape[0], new_user_sequence.shape[1], 1))
new_user_sequence

array([[[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [14]]])

In [97]:
predicted_item_id = model.predict(new_user_sequence)
predicted_item_id = label_encoder.inverse_transform([np.argmax(predicted_item_id)])
print('Recommended item:', predicted_item_id[0])

1/1 [==============================] - 0s 257ms/step
Recommended item: screen protector1


In [101]:
new_user_data = pd.DataFrame({'user_id': [4, 4], 'item_id': ['phone1', 'case1'], 'timestamp': ['2022-01-01 10:00:00', '2022-01-01 10:05:00']})
new_user_data['item_id'] = label_encoder.transform(new_user_data['item_id'])
new_user_sequence = new_user_data['item_id'].tolist()
new_user_sequence = pad_sequences([new_user_sequence], maxlen=max_sequence_length, padding='pre', truncating='pre')
new_user_sequence = new_user_sequence.reshape((new_user_sequence.shape[0], new_user_sequence.shape[1], 1))
predicted_item_id = model.predict(new_user_sequence)
predicted_item_id = label_encoder.inverse_transform([np.argmax(predicted_item_id)])
print('Recommended item:', predicted_item_id[0])

1/1 [==============================] - 0s 20ms/step
Recommended item: car charger2
